In [1]:
%load_ext autoreload
%autoreload 2

# Project Setup

In [2]:
from brollm import BaseLLM
from typing import Dict, List, Any
import httpx

class OllamaChat:
    def __init__(
            self, 
            model_name: str = "qwen3:8b",
            temperature: float = 0,
            base_url: str = "http://localhost:11434"
    ):
        self.model_name = model_name
        self.temperature = temperature
        self.base_url = base_url

    def UserMessage(self, text: str, **kwargs) -> Dict[str, Any]:
        return {"role": "user", "content": text}

    def AIMessage(self, text: str) -> Dict[str, Any]:
        return {"role": "assistant", "content": text}

    def SystemMessage(self, text: str) -> Dict[str, Any]:
        return {"role": "system", "content": text}

    def run(self, system_prompt: str, messages: List[Dict[str, Any]]) -> dict:
        all_messages = [self.SystemMessage(system_prompt)] + messages
        
        response = httpx.post(f'{self.base_url}/api/chat', json={
            "model": self.model_name,
            "messages": all_messages,
            "stream": False,
            "options": {"temperature": self.temperature},
        },
            timeout=300
        )
        
        response = response.json()
        result = {"content": response['message']['content'], "tokens":{"input":0, "output":0}}
        return result

In [3]:
ollama_model = OllamaChat(model_name="gpt-oss:20b")
ollama_model.run("you're a helpful assistant", [ollama_model.UserMessage(text="Hi")])

{'content': 'Hello! How can I help you today?',
 'tokens': {'input': 0, 'output': 0}}

In [47]:
import boto3

class BedrockOpenAI:
    def __init__(
            self, 
            model_name: str = "openai.gpt-oss-20b-1:0",
            temperature: float = 0,
            region_name: str = "us-west-2",
            aws_access_key_id: str | None = None,
            aws_secret_access_key: str | None = None,
            aws_session_token: str | None = None
    ):
        self.model_name = model_name
        self.temperature = temperature
        self.region_name = region_name
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.aws_session_token = aws_session_token

    def get_model(self, ):
        param = {
            "service_name": "bedrock-runtime",
            "region_name": self.region_name,
            "aws_access_key_id": self.aws_access_key_id,
            "aws_secret_access_key": self.aws_secret_access_key,
            "aws_session_token": self.aws_session_token,
        }
        return boto3.client(**param)

    def convert_message(self, messages):
        return [{"role": m["role"], "content": [{"text": m["content"]}]} for m in messages]

    def UserMessage(self, text: str, image_bytes=None, image_format: str | None = None, **kwargs) -> Dict[str, Any]:
        return {"role": "user", "content": text}

    def AIMessage(self, text: str) -> Dict[str, Any]:
        return {"role": "assistant", "content": text}

    def SystemMessage(self, text: str) -> List[Dict[str, str]]:
        return [{"text": text}]

    def run(self, system_prompt, messages):
        model = self.get_model()
        response = model.converse(
            modelId=self.model_name,
            messages=self.convert_message(messages),
            system=self.SystemMessage(text=system_prompt),
            inferenceConfig={
                # "maxTokens": 150, 
                "temperature": self.temperature, 
                # "topP": 0.9
            },
        )
        result = {"content":response["output"]['message']['content'][-1]['text'], "tokens": {"input":response["usage"]['inputTokens'], "output":response["usage"]['outputTokens']}}
        return result

In [48]:
bedrock_model = BedrockOpenAI(model_name="openai.gpt-oss-20b-1:0")
messages = [bedrock_model.UserMessage(text="Hi")]
response = bedrock_model.run("you're a helpful assistant", messages)
messages.append(bedrock_model.AIMessage(text=response["content"]))
messages.append(bedrock_model.UserMessage(text="What is the weather like today?"))
response = bedrock_model.run("you're a helpful assistant", messages)
messages.append(bedrock_model.AIMessage(text=response['content']))
messages


[{'role': 'user', 'content': 'Hi'},
 {'role': 'assistant', 'content': 'Hello! How can I help you today?'},
 {'role': 'user', 'content': 'What is the weather like today?'},
 {'role': 'assistant',
  'content': 'I’m not able to pull in real‑time weather data. Could you let me know which city or region you’re interested in? Then I can give you a general idea of what to expect today.'}]

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def create_mock_analytics_data(n_customers=1000, n_products=200, n_purchases=5000):
    np.random.seed(42)
    random.seed(42)
    
    # Customer DataFrame
    customers = pd.DataFrame({
        'customer_id': range(1, n_customers + 1),
        'age': np.random.randint(18, 80, n_customers),
        'gender': np.random.choice(['M', 'F'], n_customers),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_customers),
        'state': np.random.choice(['CA', 'TX', 'NY', 'FL', 'IL'], n_customers),
        'income': np.random.normal(65000, 25000, n_customers).astype(int),
        'signup_date': pd.date_range('2020-01-01', '2024-01-01', periods=n_customers),
        'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], n_customers),
        'preferred_channel': np.random.choice(['Online', 'Store', 'Mobile'], n_customers),
        'loyalty_score': np.random.randint(1, 101, n_customers)
    })
    
    # Product DataFrame
    products = pd.DataFrame({
        'product_id': range(1, n_products + 1),
        'category': np.random.choice(['Electronics', 'Clothing', 'Home', 'Sports'], n_products),
        'brand': np.random.choice(['Apple', 'Samsung', 'Nike', 'Adidas', 'IKEA'], n_products),
        'price': np.round(np.random.exponential(50, n_products) + 10, 2),
        'cost': np.round(np.random.exponential(30, n_products) + 5, 2),
        'rating': np.round(np.random.uniform(1, 5, n_products), 2),
        'stock_quantity': np.random.randint(0, 1000, n_products)
    })
    
    # Purchase DataFrame
    purchases = pd.DataFrame({
        'purchase_id': range(1, n_purchases + 1),
        'customer_id': np.random.choice(customers['customer_id'], n_purchases),
        'product_id': np.random.choice(products['product_id'], n_purchases),
        'purchase_date': pd.date_range('2020-01-01', '2024-12-01', periods=n_purchases),
        'quantity': np.random.poisson(2, n_purchases) + 1,
        'channel': np.random.choice(['Online', 'Store', 'Mobile'], n_purchases),
        'discount_applied': np.round(np.random.uniform(0, 0.3, n_purchases), 2),
        'returned': np.random.choice([True, False], n_purchases, p=[0.08, 0.92])
    })
    
    # Add total amount
    purchases = purchases.merge(products[['product_id', 'price']], on='product_id')
    purchases['total_amount'] = np.round(purchases['quantity'] * purchases['price'] * (1 - purchases['discount_applied']), 2)
    
    return customers, products, purchases

In [50]:
customers_df, products_df, purchases_df = create_mock_analytics_data()

In [5]:
customers_df

,customer_id,age,gender,city,state,income,signup_date,customer_segment,preferred_channel,loyalty_score
0,1,56,M,Phoenix,IL,43243,2020-01-01 00:00:00.000000000,Basic,Store,98
1,2,69,M,New York,FL,59657,2020-01-02 11:05:56.756756756,Basic,Mobile,77
2,3,46,F,Chicago,IL,79007,2020-01-03 22:11:53.513513513,Basic,Store,58
3,4,32,M,Los Angeles,TX,49609,2020-01-05 09:17:50.270270270,Basic,Online,78
4,5,60,M,Chicago,FL,82346,2020-01-06 20:23:47.027027027,Basic,Store,75
...,...,...,...,...,...,...,...,...,...,...
995,996,18,M,New York,TX,103389,2023-12-26 03:36:12.972972960,Standard,Store,44
996,997,35,F,Phoenix,TX,78468,2023-12-27 14:42:09.729729728,Premium,Store,64
997,998,49,F,Chicago,NY,59718,2023-12-29 01:48:06.486486480,Premium,Mobile,31
998,999,64,M,Chicago,CA,54000,2023-12-30 12:54:03.243243232,Premium,Online,47


In [6]:
products_df

,product_id,category,brand,price,cost,rating,stock_quantity
0,1,Sports,Apple,28.20,84.09,4.30,269
1,2,Clothing,Nike,19.25,7.04,3.59,291
2,3,Electronics,Nike,40.87,28.92,4.88,452
3,4,Clothing,Samsung,28.34,31.81,3.86,714
4,5,Electronics,Apple,11.79,36.73,4.70,23
...,...,...,...,...,...,...,...
195,196,Sports,Adidas,209.29,28.17,2.33,707
196,197,Home,Samsung,15.12,7.19,3.98,445
197,198,Clothing,Samsung,28.05,86.56,1.61,207
198,199,Home,IKEA,60.84,48.62,1.52,469


In [7]:
purchases_df

,purchase_id,customer_id,product_id,purchase_date,quantity,channel,discount_applied,returned,price,total_amount
0,1,976,82,2020-01-01 00:00:00.000000000,4,Mobile,0.14,False,12.58,43.28
1,2,819,131,2020-01-01 08:37:21.088217643,3,Store,0.27,False,35.75,78.29
2,3,104,118,2020-01-01 17:14:42.176435287,5,Online,0.17,False,17.29,71.75
3,4,742,81,2020-01-02 01:52:03.264652930,2,Mobile,0.04,False,48.85,93.79
4,5,426,70,2020-01-02 10:29:24.352870574,4,Store,0.16,False,41.40,139.10
...,...,...,...,...,...,...,...,...,...,...
4995,4996,91,182,2024-11-29 13:30:35.647129408,1,Store,0.13,False,30.59,26.61
4996,4997,854,16,2024-11-29 22:07:56.735347072,3,Online,0.11,False,81.26,216.96
4997,4998,272,183,2024-11-30 06:45:17.823564704,3,Store,0.07,False,10.91,30.44
4998,4999,834,94,2024-11-30 15:22:38.911782336,2,Mobile,0.10,True,24.70,44.46


# Generate metadata in metadata/*.yaml (run only once)

In [8]:
# Usage
# from broinsight.metadata.metadata_generator import DataFrameMetadataGenerator
# generator = DataFrameMetadataGenerator()
# generator.generate_metadata(customers_df, "customers_df", "Customer information with loyalty score")
# generator.generate_metadata(products_df, "products_df", "Product catalog with pricing and inventory")
# generator.generate_metadata(purchases_df, "purchases_df", "Customer purchases and return status")


# Edit description of each field in metadata/*.yaml

# Load metadata from metadata/*.yaml

In [4]:
from broinsight.metadata.metadata_loader import MetadataLoader

# Usage
loader = MetadataLoader("metadata")

# Load all tables
full_context = loader.construct_prompt_context()

# Load specific tables only
selected_context = loader.construct_prompt_context(["customers_df", "products_df"])


In [108]:
print(loader.get_summary_prompt())

Table: purchases_df - Description: **Table: Purchases and Returns**

This table records every customer order and whether the order was returned. It includes the date and time of the sale, the product bought, the quantity, the price, any discount applied, the total amount paid, the channel through which the purchase was made (online, store, mobile), and the customer who made the purchase.  

**Primary purpose / use cases**  
- **Sales performance** – view total revenue, average sale size, and top‑selling products.  
- **Channel analysis** – compare how many sales occur in each channel and how channel mix affects revenue.  
- **Return monitoring** – calculate return rates, identify high‑return products, and assess the impact of returns on cash flow.  
- **Discount effectiveness** – see how discounts influence the quantity sold and the final revenue.  
- **Revenue forecasting** – feed into budgeting and planning by linking past sales, discounts, and returns to future projections.  

**Key

In [109]:
print(full_context)

TABLE: purchases_df
DESCRIPTION: **Table: Purchases and Returns**

This table records every customer order and whether the order was returned. It includes the date and time of the sale, the product bought, the quantity, the price, any discount applied, the total amount paid, the channel through which the purchase was made (online, store, mobile), and the customer who made the purchase.  

**Primary purpose / use cases**  
- **Sales performance** – view total revenue, average sale size, and top‑selling products.  
- **Channel analysis** – compare how many sales occur in each channel and how channel mix affects revenue.  
- **Return monitoring** – calculate return rates, identify high‑return products, and assess the impact of returns on cash flow.  
- **Discount effectiveness** – see how discounts influence the quantity sold and the final revenue.  
- **Revenue forecasting** – feed into budgeting and planning by linking past sales, discounts, and returns to future projections.  

**Key r

In [110]:
print(selected_context)

TABLE: customers_df
DESCRIPTION: 
This table holds the key details about each customer, including who they are, where they live, how much they earn, and how loyal they are to the brand. It is used by marketing, sales, and product teams to segment customers, target promotions, and measure loyalty. The data links directly to customer‑service interactions, purchase history, and campaign results, helping the company tailor offers, forecast revenue, and improve retention. Typical users are marketing analysts, loyalty program managers, and senior executives who need a clear view of customer profiles and engagement levels.
FIELDS:
  - age (INTEGER): Customer age in years (18-79)
  - city (VARCHAR): Customer residential city [Values: Chicago, Houston, Los Angeles, New York, Phoenix]
  - customer_id (INTEGER): Unique customer identifier
  - customer_segment (VARCHAR): Customer tier classification (Basic, Standard, Premium) [Values: Basic, Premium, Standard]
  - gender (VARCHAR): Customer gender

# Generate table description (run only once)

In [111]:
from broinsight.metadata.table_descriptor import TableDescriptor
from broprompt import Prompt

In [112]:
td = TableDescriptor(
    Prompt.from_markdown("./broinsight/prompt_hub/table_descriptor.md").str, 
    OpenAI20b()
)

In [114]:
metadata_dict = loader.get_metadata_dict()
for table_name, detail in metadata_dict.items():
    description = td.run("{metadata}".format(metadata=detail))
    metadata_dict[table_name]['description'] = description

In [115]:
import yaml

for table_name, detail in metadata_dict.items():
    with open(f"metadata/{table_name}.yaml", "w") as f:
        detail.update({"table_name": table_name})
        yaml.dump(detail, f)

# Initialize Connector

In [51]:
from broinsight.metadata.metadata_db import DuckConnector
connector = DuckConnector()
connector.register_dataframe(customers_df, "customers_df")
connector.register_dataframe(products_df, "products_df")
connector.register_dataframe(purchases_df, "purchases_df")
connector.register_metadata_from_yaml("metadata/")

In [52]:
print(connector.construct_prompt_context())

TABLE: purchases_df
DESCRIPTION: **Table: Purchases and Returns**

This table logs every customer order, including the product bought, quantity, price, any discount, the final amount paid, the sales channel (online, store, or mobile), the customer who made the purchase, and whether the order was returned.

**Why it matters**

- **Track sales performance** – see total revenue, average sale size, and which products sell best.  
- **Compare channels** – understand how online, in‑store, and mobile sales differ and how each channel contributes to revenue.  
- **Monitor returns** – calculate return rates, spot high‑return products, and assess the cash‑flow impact of returns.  
- **Evaluate discounts** – measure how discounts affect quantity sold and net revenue.  
- **Support forecasting** – feed past sales, discounts, and return data into budgeting and revenue projections.

**Key connections**

- *Customer* – links each transaction to a customer profile for segmentation.  
- *Product* – tie

# Start BroInsight

In [54]:
from broinsight import BroInsight, Shared
from broflow import state

state.set("debug", True)
# model = OllamaChat(model_name="gpt-oss:20b")
model = BedrockOpenAI(model_name="openai.gpt-oss-20b-1:0")


In [72]:
# user_input = "Who is the most frequent buyer and which product category he/she buys?"
user_input = "Are there products with high sales but low remaining stock, and what is the projected stock-out risk?"
# user_input = "What is the total revenue and average order value by channel for the last 30 days?"
# user_input = "What data do we have and what should be the first question for me?"
messages = [model.UserMessage(text=user_input)]
shared = Shared(
    model=model,
    messages=messages,
    db=connector,
    metadata_db=connector
)

In [73]:
result = BroInsight.chat(shared)
shared.input_token, shared.output_token

Welcome to BroInsight!
Running action: Organize
Running action: SelectMetadata
Running action: GenerateSQL
Running action: Retrieve
Running action: Chat
Thank you for using BroInsight!


(4754, 2378)

In [74]:
print(result.messages[-1]['content'])

**Short answer:**  
Yes – all three products in the snapshot show strong sales relative to the stock they still have on hand, and each is flagged as **High** risk for running out of inventory.

| Product | Total sold | Stock left | Avg. daily sales | Days until stock‑out | Stock‑out risk |
|---------|------------|------------|------------------|----------------------|----------------|
| 92 (IKEA, Electronics) | 57 units | 6 units | 2.85 units/day | **≈ 2.1 days** | **High** |
| 170 (Apple, Home) | 32 units | 8 units | 3.20 units/day | **≈ 2.5 days** | **High** |
| 185 (Apple, Electronics) | 72 units | 13 units | 3.00 units/day | **≈ 4.3 days** | **High** |

### What this means

- **High sales volume**: Each product has sold a sizable number of units (57, 32, and 72 respectively) in the period tracked.
- **Low remaining stock**: The on‑hand inventory (6, 8, and 13 units) is small relative to the sales pace.
- **Short time to depletion**: The projected days until stock‑out are all under 

In [75]:
print(result.sql_query)


WITH sales AS (
    SELECT
        p.product_id,
        SUM(p.quantity)          AS total_quantity_sold,
        SUM(p.total_amount)      AS total_revenue,
        COUNT(DISTINCT DATE(p.purchase_date)) AS sales_days
    FROM purchases_df p
    WHERE p.returned = FALSE
    GROUP BY p.product_id
),
product_stats AS (
    SELECT
        s.product_id,
        s.total_quantity_sold,
        s.total_revenue,
        s.sales_days,
        s.total_quantity_sold / NULLIF(s.sales_days, 0) AS avg_daily_sales,
        pr.stock_quantity,
        pr.brand,
        pr.category,
        pr.stock_quantity / NULLIF(s.total_quantity_sold / NULLIF(s.sales_days, 0), 0) AS projected_days_until_stock_out
    FROM sales s
    JOIN products_df pr
      ON s.product_id = pr.product_id
)
SELECT
    product_id,
    brand,
    category,
    total_quantity_sold,
    total_revenue,
    stock_quantity,
    ROUND(avg_daily_sales, 2)                     AS avg_daily_sales,
    ROUND(projected_days_until_stock_out, 2)

In [76]:
connector.execute_query(result.sql_query)

,product_id,brand,category,total_quantity_sold,total_revenue,stock_quantity,avg_daily_sales,projected_days_until_stock_out,stock_out_risk
0,92,IKEA,Electronics,57.0,1183.66,6,2.85,2.11,High
1,170,Apple,Home,32.0,3647.72,8,3.20,2.50,High
2,185,Apple,Electronics,72.0,2980.15,13,3.00,4.33,High


In [70]:
shared.session_logger.get_summary()

{'total_actions': 5,
 'actions': ['Organize', 'SelectMetadata', 'GenerateSQL', 'Retrieve', 'Chat'],
 'errors': []}

In [71]:
shared.session_logger.get_session_logs()

[{'timestamp': '2025-09-05T10:50:46.818981',
  'action': 'Organize',
  'input_data': 'METADATAS:\n\nTable: purchases_df - Description: **Table: Purchases and Returns**\n\nThis table logs every customer order, including the product bought, quantity, price, any discount, the final amount paid, the sales channel (online, store, or mobile), the customer who made the purchase, and whether the order was returned.\n\n**Why it matters**\n\n- **Track sales performance** – see total revenue, average sale size, and which products sell best.  \n- **Compare channels** – understand how online, in‑store, and mobile sales differ and how each channel contributes to revenue.  \n- **Monitor returns** – calculate return rates, spot high‑return products, and assess the cash‑flow impact of returns.  \n- **Evaluate discounts** – measure how discounts affect quantity sold and net revenue.  \n- **Support forecasting** – feed past sales, discounts, and return data into budgeting and revenue projections.\n\n**Ke

In [96]:
shared.retries

0